In [2]:
import easydict
import json

In [90]:
args = easydict.EasyDict()

args.default_path = "../data/encore21titanic/"
args.train_csv = args.default_path+"train.csv"
args.test_csv = args.default_path+"test.csv"
args.submission_csv = args.default_path+"submission.csv"

args.random_state = 21

In [91]:
import pandas as pd

In [92]:
train = pd.read_csv(args.train_csv)
print(train.shape)
train.head()

(916, 12)


,passengerid,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,0,0,2,"Wheeler, Mr. Edwin Frederick""""",male,NaN,0,0,SC/PARIS 2159,12.8750,NaN,S
1,1,0,3,"Henry, Miss. Delia",female,NaN,0,0,382649,7.7500,NaN,Q
2,2,1,1,"Hays, Mrs. Charles Melville (Clara Jennings Gr...",female,52.0,1,1,12749,93.5000,B69,S
3,3,1,3,"Andersson, Mr. August Edvard (""Wennerstrom"")",male,27.0,0,0,350043,7.7958,NaN,S
4,4,0,2,"Hold, Mr. Stephen",male,44.0,1,0,26707,26.0000,NaN,S


In [93]:
test = pd.read_csv(args.test_csv)
print(test.shape)
test.head()

(393, 11)


,passengerid,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,916,3,"McGowan, Miss. Anna ""Annie""",female,15.0,0,0,330923,8.0292,NaN,Q
1,917,2,"Pinsky, Mrs. (Rosa)",female,32.0,0,0,234604,13.0000,NaN,S
2,918,3,"McCarthy, Miss. Catherine Katie""""",female,NaN,0,0,383123,7.7500,NaN,Q
3,919,3,"Franklin, Mr. Charles (Charles Fardon)",male,NaN,0,0,SOTON/O.Q. 3101314,7.2500,NaN,S
4,920,1,"Wick, Mrs. George Dennick (Mary Hitchcock)",female,45.0,1,1,36928,164.8667,NaN,S


In [94]:
submission = pd.read_csv(args.submission_csv)
print(submission.shape)
submission

(393, 2)


,passengerid,survived
0,916,0.5
1,917,0.5
2,918,0.5
3,919,0.5
4,920,0.5
...,...,...
388,1304,0.5
389,1305,0.5
390,1306,0.5
391,1307,0.5


In [95]:
ori_train = pd.read_csv(args.train_csv)
ori_test = pd.read_csv(args.test_csv)

ori_test.isnull().sum()

passengerid      0
pclass           0
name             0
gender           0
age             83
sibsp            0
parch            0
ticket           0
fare             1
cabin          296
embarked         1
dtype: int64

In [96]:
age_mean = ori_train['age'].mean()
fare_mean = ori_train['fare'].mean()
cabin_mode = ori_train['cabin'].mode().values[0]
embarked_mode = ori_train['embarked'].mode().values[0]

age_mean, fare_mean, cabin_mode, embarked_mode

(29.69836956521739, 32.40271048034934, 'B57 B59 B63 B66', 'S')

In [97]:
ori_train['cabin'].loc[ori_train['cabin'] == "G6"]
# describe()
# loc[ori_train['cabin'] == "B57 B59 B63 B66"]

113    G6
549    G6
731    G6
815    G6
Name: cabin, dtype: object

In [98]:
ori_train['age'].fillna(age_mean, inplace=True)
ori_train['fare'].fillna(fare_mean, inplace=True)
ori_train['cabin'].fillna(cabin_mode, inplace=True)
ori_train['embarked'].fillna(embarked_mode, inplace=True)
ori_test['age'].fillna(age_mean, inplace=True)
ori_test['fare'].fillna(fare_mean, inplace=True)
ori_test['cabin'].fillna(cabin_mode, inplace=True)
ori_test['embarked'].fillna(embarked_mode, inplace=True)

ori_train.isnull().sum().sum(), ori_test.isnull().sum().sum()

(0, 0)

In [99]:
pd.DataFrame(["male", "female", "male", "male"]).replace({"male": 1, "female": 0})  

,0
0,1
1,0
2,1
3,1


In [100]:
ori_train['gender'].replace({"male": 1, "female": 0}, inplace=True)
ori_test['gender'].replace({"male": 1, "female": 0}, inplace=True)

In [248]:
ori_train["embarked"].replace({"S":0, "Q":1, "C":2}, inplace=True)
ori_test["embarked"].replace({"S":0, "Q":1, "C":2}, inplace=True)

In [112]:
import re

In [172]:
alb = re.compile("[^0-9]")
tmp = "B57 B59 B63 B66"
# for x in alb.findall(tmp):
#     tmp = tmp.replace(x, "")
# tmp
alb.sub("", tmp)

'57596366'

In [178]:
len(alb.findall("123"))

0

In [238]:
ori_train["cabin_sub"] = list(map(lambda x: alb.sub("", x), ori_train["cabin"]))
ori_test["cabin_sub"] = list(map(lambda x: alb.sub("", x), ori_test["cabin"]))
ori_train.loc[ori_train["cabin_sub"] == "", "cabin_sub"] = 0
ori_test.loc[ori_test["cabin_sub"] == "", "cabin_sub"] = 0

In [183]:
ori_train["ticket_sub"] = list(map(lambda x: alb.sub("", x), ori_train["ticket"]))

,passengerid,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked,ticket_sub
51,51,0,3,"Johnson, Mr. William Cahoone Jr",1,19.0,0,0,LINE,0.0,B57 B59 B63 B66,S,
158,158,0,3,"Johnson, Mr. Alfred",1,49.0,0,0,LINE,0.0,B57 B59 B63 B66,S,
187,187,1,3,"Tornquist, Mr. William Henry",1,25.0,0,0,LINE,0.0,B57 B59 B63 B66,S,
500,500,0,3,"Leonard, Mr. Lionel",1,36.0,0,0,LINE,0.0,B57 B59 B63 B66,S,


In [226]:
ori_test["ticket_sub"] = list(map(lambda x: alb.sub("", x), ori_test["ticket"]))

In [189]:
ori_train.loc[ori_train["ticket_sub"] == ""]

,passengerid,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked,ticket_sub
51,51,0,3,"Johnson, Mr. William Cahoone Jr",1,19.0,0,0,LINE,0.0,B57 B59 B63 B66,S,
158,158,0,3,"Johnson, Mr. Alfred",1,49.0,0,0,LINE,0.0,B57 B59 B63 B66,S,
187,187,1,3,"Tornquist, Mr. William Henry",1,25.0,0,0,LINE,0.0,B57 B59 B63 B66,S,
500,500,0,3,"Leonard, Mr. Lionel",1,36.0,0,0,LINE,0.0,B57 B59 B63 B66,S,


In [206]:
ori_train.loc[ori_train["ticket_sub"] == "", "ticket_sub"] = 0

In [227]:
ori_test.loc[ori_test["ticket_sub"] == "", "ticket_sub"] = 0

In [214]:
ori_train.loc[ori_train["passengerid"] == 158]

,passengerid,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked,ticket_sub
158,158,0,3,"Johnson, Mr. Alfred",1,49.0,0,0,LINE,0.0,B57 B59 B63 B66,S,0


In [215]:
ori_train["ticket"].nunique()

703

In [249]:
cols = ['pclass', 'age', 'sibsp', 'parch', 'fare', 'gender', 'cabin_sub', 'embarked', "ticket_sub"]
X = ori_train[cols]
y = ori_train['survived']
test = ori_test[cols + ['passengerid']]

X.shape, y.shape, test.shape

((916, 9), (916,), (393, 10))

In [250]:
from sklearn.model_selection import train_test_split

In [251]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, stratify=ori_train["survived"], random_state=args.random_state)

In [252]:
X_tr.isnull().sum().sum(), X_te.isnull().sum().sum(), test.isnull().sum().sum()

(0, 0, 0)

In [253]:
X_tr.shape, X_te.shape, test.shape

((732, 9), (184, 9), (393, 10))

In [254]:
test.set_index(['passengerid'], inplace=True)

test.head()

,pclass,age,sibsp,parch,fare,gender,cabin_sub,embarked,ticket_sub
passengerid,,,,,,,,,
916,3,15.00000,0,0,8.0292,0,57596366,1,330923
917,2,32.00000,0,0,13.0000,0,57596366,0,234604
918,3,29.69837,0,0,7.7500,0,57596366,1,383123
919,3,29.69837,0,0,7.2500,1,57596366,0,3101314
920,1,45.00000,1,1,164.8667,0,57596366,0,36928


In [255]:
from sklearn.tree import DecisionTreeClassifier

In [256]:
modelV0 = DecisionTreeClassifier(random_state=args.random_state)

In [257]:
modelV0.fit(X_tr, y_tr)

DecisionTreeClassifier(random_state=21)

In [258]:
from sklearn.metrics import roc_curve, auc

In [259]:
pred = modelV0.predict_proba(X_te)[:,1]
fpr, tpr, _ = roc_curve(y_te, pred)

In [260]:
auc(fpr, tpr)

0.762781954887218

In [261]:
test_pred = modelV0.predict_proba(test)[:,1]
test_pred.shape

(393,)

In [262]:
submission["survived"] = test_pred
submission

,passengerid,survived
0,916,1.0
1,917,1.0
2,918,1.0
3,919,0.0
4,920,1.0
...,...,...
388,1304,1.0
389,1305,1.0
390,1306,1.0
391,1307,0.0


In [263]:
submission.set_index("passengerid")
submission

,passengerid,survived
0,916,1.0
1,917,1.0
2,918,1.0
3,919,0.0
4,920,1.0
...,...,...
388,1304,1.0
389,1305,1.0
390,1306,1.0
391,1307,0.0


In [264]:
submission.to_csv("../data/encore21titanic/submission_0220_2.csv", index=False, header=True)